In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_train_path="/content/drive/MyDrive/AI-datasets/Ball/BallData/train"
data_test_path="/content/drive/MyDrive/AI-datasets/Ball/BallData/test"

batch_size=4
img_size=224
optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01,decay = 1e-4)
loss="categorical_crossentropy"
epochs=10
onnx_path="/content/drive/MyDrive/AI-datasets/Ball/BallData/Onnx/model.onnx"


In [ ]:
image_generator_test = ImageDataGenerator()    
data_train = image_generator_test.flow_from_directory(batch_size=batch_size,
                                                  directory=data_train_path,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                  class_mode='categorical'
  
                                                  )
data_test = image_generator_test.flow_from_directory(batch_size=batch_size,
                                                  directory=data_test_path,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                  class_mode='categorical'
  
                                                  )

Found 172 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
print("Train Data",data_train.class_indices)
print("Test Data",data_test.class_indices)



Train Data {'colored_beachball': 0, 'football': 1}
Test Data {'colored_beachball': 0, 'football': 1}


In [ ]:
vgg=tf.keras.applications.VGG16(
        include_top=False,
        weights="imagenet",
        classes=4,
    )

vgg.trainable=False
inputs=tf.keras.layers.Input(shape=(img_size,img_size,3))
x=tf.keras.layers.Rescaling(scale=1./255.)(inputs)
x=vgg(x)
x=tf.keras.layers.GlobalMaxPool2D()(x)
x=tf.keras.layers.Dense(128,activation="relu")(x)
x=tf.keras.layers.Dropout(0.25)(x)
x=tf.keras.layers.Dense(len(data_train.class_indices),activation="softmax")(x)
model=tf.keras.Model(inputs=inputs,outputs=x)
model.summary()

model.compile(optimizer =optimizer,loss = loss,  metrics = ["accuracy"])


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_max_pooling2d (Globa  (None, 512)              0         
 lMaxPooling2D)                                                  
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dropout (Dropout)           (None, 128)               0      

In [ ]:
callbacks=[]
callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss',factor = 0.1,patience = 2,min=1e-6,verbose=1))
callbacks.append(tf.keras.callbacks.EarlyStopping(monitor="loss",patience=4,restore_best_weights=True))

In [ ]:
model.fit(data_train,validation_data=data_test,  epochs = epochs,callbacks=callbacks)


Epoch 1/10
36/43 [========================>.....] - ETA: 17s - loss: 1.3423 - accuracy: 0.7778

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


43/43 [==============================] - 119s 3s/step - loss: 1.1741 - accuracy: 0.8023 - val_loss: 0.0018 - val_accuracy: 1.0000 - lr: 0.0100
Epoch 2/10
43/43 [==============================] - 117s 3s/step - loss: 0.2446 - accuracy: 0.9302 - val_loss: 0.0304 - val_accuracy: 1.0000 - lr: 0.0100
Epoch 3/10
43/43 [==============================] - 109s 3s/step - loss: 0.1294 - accuracy: 0.9767 - val_loss: 0.0108 - val_accuracy: 1.0000 - lr: 0.0100
Epoch 4/10
43/43 [==============================] - 112s 3s/step - loss: 0.0290 - accuracy: 0.9942 - val_loss: 0.0022 - val_accuracy: 1.0000 - lr: 0.0100
Epoch 5/10
43/43 [==============================] - 109s 3s/step - loss: 0.0254 - accuracy: 0.9942 - val_loss: 0.1036 - val_accuracy: 0.9000 - lr: 0.0100
Epoch 6/10
43/43 [==============================] - 111s 3s/step - loss: 0.1077 - accuracy: 0.9767 - val_loss: 0.0015 - val_accuracy: 1.0000 - lr: 0.0100
Epoch 7/10
43/43 [==============================] - ETA: 0s - loss: 0.0300 - accuracy: 

In [ ]:
from PIL import Image
import numpy as np

img = Image.open("/content/drive/MyDrive/AI-datasets/Ball/BallTest/easy-football.png").convert('RGB').resize((img_size,img_size))
img=np.asarray(img)
predicted=model.predict(np.expand_dims(img,axis=0))
print(list(data_train.class_indices.keys())[np.argmax(predicted)])
print(predicted)

1/1 [==============================] - 1s 663ms/step
football
[[5.1110535e-04 9.9948883e-01]]


In [ ]:
from tensorflow.python.framework import convert_to_constants
save_model_path="/content/drive/MyDrive/AI-datasets/Ball/BallData/Onnx"

@tf.function(input_signature=[tf.TensorSpec(shape=[None,224,224,3], dtype=tf.float32)])
def to_save(x):
    return model(x)
f = to_save.get_concrete_function()
constantGraph = convert_to_constants.convert_variables_to_constants_v2(f)
tf.io.write_graph(constantGraph.graph.as_graph_def(), "/content/drive/MyDrive/AI-datasets/Ball/BallData/Onnx_Graph", 'model.pb') 

'/content/drive/MyDrive/AI-datasets/Ball/BallData/Onnx_Graph/model.pb'

In [ ]:
import os
save_model_path="/content/drive/MyDrive/AI-datasets/Ball/BallData/Onnx"
if not os.path.isdir(save_model_path):
      os.makedirs(save_model_path)

model.save(save_model_path)